### Vector Autoregressive

In [ ]:
# Import the required packages
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller

# Load the data
data = pd.read_csv("data.csv", index_col=0)

# Convert the data to a time series object
ts_data = data.iloc[:,1:4]
ts_data.index = pd.to_datetime(ts_data.index)
ts_data = ts_data.asfreq('MS')

# Split the data into training and test sets
train_data = ts_data.loc[:'2015-12-01']
test_data = ts_data.loc['2016-01-01':]

# Perform an Augmented Dickey-Fuller test on each variable to check for stationarity
adf_test = adfuller(train_data, maxlag=1)
adf_test_df = pd.DataFrame(adf_test[0:2], index=['Test Statistic', 'p-value'], columns=['Value'])
adf_test_df.index.name = 'Augmented Dickey-Fuller Test'
adf_test_df.columns.name = 'Variable'
print(adf_test_df)

# Estimate the VAR model with 2 lags using the training data
var_model = VAR(train_data)
var_result = var_model.fit(2)

# Print the summary of the VAR model
print(var_result.summary())

# Plot the lag order selection criterion
print(var_result.plot_diagnostics())

# Forecast future values of the first variable in the VAR model using the test data
forecast = var_result.forecast(train_data.values[-2:], len(test_data))

# Calculate the Mean Absolute Error (MAE) of the forecast
mae = np.mean(np.abs(forecast[:,0] - test_data.values[:,0]))

# Plot the forecast and actual values
import matplotlib.pyplot as plt
plt.plot(test_data.iloc[:,0], label='Actual')
plt.plot(forecast[:,0], label='Forecast', linestyle='--')
plt.legend()

# Print the MAE of the forecast
print("Mean Absolute Error:", round(mae, 2))
